# 쿼리
- 쿼리, 가져오기, 쿼리이터레이터를 사용하여 메타데이터 필터링을 수행하는 방법 소개

참고자료: https://milvus.io/docs/ko/get-and-scalar-query.md

## 개요
- 컬렉션에는 다양한 유형의 스칼라 필드를 저장할 수 있음
- 하나 이상의 스칼라 필드를 기반으로 Milvus가 엔티티를 필터링하도록 할 수 있음

## Get 사용
- 기본 키로 엔티티를 찾아야 하는 경우 Get 메서드를 사용할 수 있음

In [5]:
data = [
        {"id": 0, "vector": [0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592], "color": "pink_8682"},
        {"id": 1, "vector": [0.19886812562848388, 0.06023560599112088, 0.6976963061752597, 0.2614474506242501, 0.838729485096104], "color": "red_7025"},
        {"id": 2, "vector": [0.43742130801983836, -0.5597502546264526, 0.6457887650909682, 0.7894058910881185, 0.20785793220625592], "color": "orange_6781"},
        {"id": 3, "vector": [0.3172005263489739, 0.9719044792798428, -0.36981146090600725, -0.4860894583077995, 0.95791889146345], "color": "pink_9298"},
        {"id": 4, "vector": [0.4452349528804562, -0.8757026943054742, 0.8220779437047674, 0.46406290649483184, 0.30337481143159106], "color": "red_4794"},
        {"id": 5, "vector": [0.985825131989184, -0.8144651566660419, 0.6299267002202009, 0.1206906911183383, -0.1446277761879955], "color": "yellow_4222"},
        {"id": 6, "vector": [0.8371977790571115, -0.015764369584852833, -0.31062937026679327, -0.562666951622192, -0.8984947637863987], "color": "red_9392"},
        {"id": 7, "vector": [-0.33445148015177995, -0.2567135004164067, 0.8987539745369246, 0.9402995886420709, 0.5378064918413052], "color": "grey_8510"},
        {"id": 8, "vector": [0.39524717779832685, 0.4000257286739164, -0.5890507376891594, -0.8650502298996872, -0.6140360785406336], "color": "white_9381"},
        {"id": 9, "vector": [0.5718280481994695, 0.24070317428066512, -0.3737913482606834, -0.06726932177492717, -0.6980531615588608], "color": "purple_4976"},
]

In [8]:
from pymilvus import MilvusClient, DataType

# 클라이언트 연결
client = MilvusClient(uri="http://localhost:19530")

if client.has_collection("my_collection"):
    client.drop_collection("my_collection")

# 정확하게 정의된 스키마
schema = client.create_schema(
    auto_id=False,
    enable_dynamic_schema=True
)

schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True, auto_id=False)
schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=5)
schema.add_field(field_name="color", datatype=DataType.VARCHAR, max_length=32)

index_params = client.prepare_index_params()
index_params.add_index(field_name="vector", index_type="AUTOINDEX", metric_type="IP")

# 컬렉션 생성
client.create_collection(
    collection_name="my_collection",
    vector_field_name="vector",
    schema=schema,
    index_params=index_params
)

In [9]:
client.insert(
    collection_name="my_collection",
    data=data
)

{'insert_count': 10, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'cost': 0}

In [ ]:
res = client.get(
    collection_name="my_collection",
    ids=[0, 1, 2],
    output_fields=["vector", "color"]
)

print(res)


data: ["{'vector': [np.float32(0.35803765), np.float32(-0.6023496), np.float32(0.18414013), np.float32(-0.26286206), np.float32(0.90294385)], 'color': 'pink_8682', 'id': 0}", "{'vector': [np.float32(0.19886813), np.float32(0.060235605), np.float32(0.6976963), np.float32(0.26144746), np.float32(0.8387295)], 'color': 'red_7025', 'id': 1}", "{'vector': [np.float32(0.43742132), np.float32(-0.55975026), np.float32(0.6457888), np.float32(0.7894059), np.float32(0.20785794)], 'color': 'orange_6781', 'id': 2}"]


## 쿼리 사용

In [13]:
res = client.query(
    collection_name="my_collection",
    filter="color like \"red%\"",
    output_fields=["vector", "color"],
    limit=3
)

print(res)

data: ["{'vector': [np.float32(0.19886813), np.float32(0.060235605), np.float32(0.6976963), np.float32(0.26144746), np.float32(0.8387295)], 'color': 'red_7025', 'id': 1}", "{'vector': [np.float32(0.44523495), np.float32(-0.8757027), np.float32(0.82207793), np.float32(0.4640629), np.float32(0.3033748)], 'color': 'red_4794', 'id': 4}", "{'vector': [np.float32(0.8371978), np.float32(-0.015764369), np.float32(-0.31062937), np.float32(-0.56266695), np.float32(-0.8984948)], 'color': 'red_9392', 'id': 6}"]


## 쿼리 이터레이터 사용

In [14]:
from pymilvus import connections, Collection

connections.connect(
    uri="http://localhost:19530",
    token="root:Milvus"
)

collection = Collection("my_collection")

iterator = collection.query_iterator(
    batch_size=10,
    expr="color like \"red%\"",
    output_fields=["color"]
)

results = []

while True:
    result = iterator.next()
    if not result:
        iterator.close()
        break

    print(result)
    results += result

[{'color': 'red_7025', 'id': 1}, {'color': 'red_4794', 'id': 4}, {'color': 'red_9392', 'id': 6}]
